In [1]:
from selenium import webdriver
import time
import re
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
host = 'https://trends.google.com.tw/trends/hottrends#pn=p12'
chrome_path="C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe"#告訴webdriver chrome位置，這樣可以執行瀏覽器
browser = webdriver.Chrome(chrome_path)#開起瀏覽器
browser.get(host)#連結到想要的網址

In [ ]:
count = 1 
while count < 2310 :#因為google trends是下拉式網頁，原本預想說可以利用環圈重複執行下拉的動作直到網頁載入我們所想要的年份
                    #，結果google有限制，不給拉太多頁，所以GG
    try:
        browser.find_element_by_tag_name('body').send_keys(Keys.END)#執行下拉
        time.sleep(5)
        browser.find_element_by_class_name('more-link').click()#點擊加載
        time.sleep(15)
        count+=1
        element = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CLASS, "more-link")))#下這行會讓pythin知道當
                                                  #加載還沒家載好時不要執行下拉跟點擊的動作 
    except:
        continue

In [ ]:
soup = BeautifulSoup(browser.page_source, 'lxml')#呼叫當頁的HTML

In [ ]:
text = soup.select('div.hottrends-trends-list-container')#抓到整個內文內容

In [ ]:
with open('GoogleTrends.csv','w',newline = '') as f:
    for i in range(0,len(text)):
        content = text[i]#每當頁面加載完後新的內容會被包在div.hottrends-trends-list-container加進來
        for k in range(0,len(content)): 
            date = content.select('div.hottrends-trends-list-date-header')[k].text#將日期抓出來
            outside = content.select('div.hottrends-trends-list-date-container')[k]#當日排名的內容
            for j in range(0,(len(outside)-1)):
                datas=[]
                inside = outside.select('div > div.hottrends-single-trend-container')[j]#將排名的內容個別抓出
                rank =  inside.select('span')[0].text
                title = inside.select('span')[2].text
                times = inside.select('span.hottrends-single-trend-info-line-number')[0].text
                web = inside.select('a')[0]['href']
                try:#因為相關搜索不一定每個都有，故在此做例外處理
                    related = inside.select('span.hottrends-single-trend-related-searches-list')[0].text
                    data = date,rank,title,times,web,related
                    print(data)
                    datas.append(data)
                    csv.writer(f).writerows(datas)
                    #w.writerows(datas)
                except:
                    null = 'NULL'
                    data = date,rank,title,times,web,null
                    print(data)
                    datas.append(data)
                    csv.writer(f).writerows(datas)
                    #w.writerows(datas)
                    continue              